In [64]:
#MLFA Assignment 3
#Sanskar Singh
#19CH10047


# Importing important libraries and reading in the training data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#LOADING IN DATA FOR PART 1

#This will opan the file and save its data in our variables
with open('traindata.csv', encoding="utf8") as csv_file:
    train_raw = csv.reader(csv_file, delimiter=',')
    next(train_raw)
    
    train_dict = {}  #Dictionary containing text with category names as keys 
    categories = {}  #Dictionary containing the names of the categories and their no of occurences
    
    for line in train_raw:
        if line[0] not in train_dict:
            categories[line[0]] = 0
            train_dict[line[0]]  = ''
        train_dict[line[0]] += line[1]
        
        categories[line[0]] += 1
    
print(categories)

{'science': 20, 'sports': 20, 'business': 19, 'covid': 21}


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Functions for both parts:

In [65]:
# IMPORTANT FUNCTIONS WHICH WILL BE USED LATER



# DEFINING PREPROCESSING FUNCTION FOR TEXTUAL DATA
def text_processor(text):
    #stop word in english
    stop_words = stopwords.words('english')
    ps = PorterStemmer()
    
    #Split the words
    text = text.lower()
    words = text.split()
    #Strip the punctiations
    table = str.maketrans('', '', string.punctuation)
    
    new_words = []
    for word in words:
        if word not in stop_words:
            new_word = word.translate(table)
            new_word = ps.stem(new_word)
            new_words.append(new_word)
            
    return new_words

# Part [1(i)] Construct the vocabulary without stop-words (Stop words will not be included using above function first)
# DEFINING FUNCTION TO GENERATE FREQUENCIES OF VALUES IN A LIST

def vocabulary_generate(word_list):                 # We have already processed our text using the above function
    vocabulary = {}                                 #This function takes in text and converts it into a dictionary
    for word in word_list:                          # containing frequency of the words
        if word  not in vocabulary:
            vocabulary[word] = 0
        vocabulary[word] += 1
    return vocabulary

#Part [1(ii)] Calculate the prior distribution of the labels 
# DEFINING FUNCTION TO GENERATE PROBABILITIES FROM FREQUENCIES

def freq2probability(frequency_dict):               #In this function we convert a dictionary of frequencies
    probabilities = {}                              #into a dictionary of probabilities, helpful in finding prior probs
    freq_sum = 0
    for frequency in frequency_dict.values():
        freq_sum += frequency
    for key,frequency in frequency_dict.items():
        probabilities[key] = frequency/freq_sum
    return probabilities


#Part[1(iii)]Calculate the class-conditional probabilities of each word in the vocabulary, for each topic 
def library_constructor(train_dict):
    mega_lib = {}
    for category,text in train_dict.items():                 #In this function we take in our training data and
         text_words = text_processor(text)                   # calculate and store all the class conditional probabilities
         freq_dict =  vocabulary_generate(text_words)
         mega_lib[category] = freq2probability(freq_dict)
    return mega_lib


mega_lib = {}
mega_lib = library_constructor(train_dict)

print(mega_lib['science']['space'])                  # This value is only being printed to demonstrate
print('This was a demonstration, refer code')        # how we have saved the class conditional probabilities
                                                     # P(Xi=space| Y=science) is saved in the mega_lib as shown
                                                     # The mega_lib thus contains all our class conditional probs in a similar manner

        
        
        
# Part [1(iv)] For each test sentence, create the posterior distribution over the labels 
#NAIVE BAYES CLASSIFIER
def naive_classifier(categories,mega_lib,test_sentence):          # This function takes in the mega_lib and outputs classification 
    test_words = text_processor(test_sentence)                    # uses naives bayes classification
    confidence = {}
    for category,category_priorprob in categories.items():
        confidence[category] = category_priorprob
        for test_word in test_words:
            if test_word in mega_lib[category].keys():
                confidence[category] *= mega_lib[category][test_word]
            else:
                confidence[category] *= 0.0005
    K_inverse = 0
    for con_value in confidence.values():
          K_inverse += con_value
    for category,con_value2 in confidence.items():
         confidence[category]= con_value2/K_inverse
    return confidence




0.03980099502487562
This was a demonstration, refer code


## PART 1

In [66]:
#LOADING IN DATA FOR PART 1 FROM OUR TESTING SET

test_lines_list = []
test_lines_dict = {}

with open('testdata.csv', encoding="utf8") as csv_file2:
    test_raw = csv.reader(csv_file2, delimiter=',')
    next(test_raw)
   #Dictionary containing text with category names as keys 
    
    for line in test_raw:
           test_lines_list.append(line[1])
           test_lines_dict[line[1]]  = line[0]


In [67]:
# TABULATING OUR PREDICTIONS VS ACTUAL VALUES IN THE TRAINING DATA

table_row =[]
table_rows = []


table_header = ['Sentence']
for category in categories:
    table_header.append('Confidence in '+ str(category).title())
table_header.append('Prediction')
table_header.append('Actual')

for test_sentence in test_lines_list:
    
    table_row =[]
    
    table_row.append(test_sentence)
    confidences = naive_classifier(categories, mega_lib,test_sentence)
    for category in categories:
        table_row.append(confidences[category])
    table_row.append(max(confidences, key =confidences.get))
    table_row.append(test_lines_dict[test_sentence]) 
    table_rows.append(table_row)
    
data_frame = pd.DataFrame(table_rows,columns = table_header)
data_frame


    

,Sentence,Confidence in Science,Confidence in Sports,Confidence in Business,Confidence in Covid,Prediction,Actual
0,"He estimates that 1,000-micrometer pellets cou...",1.000000e+00,1.073310e-09,1.019644e-09,2.590748e-08,science,science
1,“That’s enough time to potentially get to Mars...,8.021422e-01,8.172842e-05,1.960943e-01,1.681755e-03,science,science
2,How exactly clumps of microbes might get expel...,9.998765e-01,3.541752e-07,1.203308e-04,2.849685e-06,science,science
3,The microbes might get kicked up by small mete...,1.000000e+00,1.806783e-11,1.089589e-09,1.897122e-11,science,science
4,"Someday, if microbial life is ever discovered ...",9.427356e-01,3.173955e-03,3.015257e-03,5.107514e-02,science,science
5,"In tennis, After the service has been correctl...",5.715968e-18,1.000000e+00,2.114963e-14,3.577307e-19,sports,sports
6,This may occur if a tennis player fails to hit...,3.012123e-19,1.000000e+00,9.173428e-20,2.447843e-20,sports,sports
7,"To win a game, a tennis player must win four p...",4.300483e-14,1.000000e+00,1.303197e-13,1.738730e-13,sports,sports
8,"In tennis, It never has been satisfactorily ex...",3.615929e-04,9.996021e-01,1.726154e-05,1.907855e-05,sports,sports
9,"In tennis, The server’s score is called first;...",1.523936e-10,9.999685e-01,3.145206e-05,1.232286e-10,sports,sports


In [68]:
# Calculating the accuracy of our predictions

correct = 0

for sentence in test_lines_list:
    prediction_dict = naive_classifier(categories, mega_lib, sentence)
    prediction = max(prediction_dict, key = prediction_dict.get)
    if prediction == test_lines_dict[sentence]:
        correct += 1
        
accuracy = 100*(correct/len(test_lines_list))

print("The accuracy of our model determined from the testing data is \n" + str(accuracy) +'%' )
        
      

The accuracy of our model determined from the testing data is 
100.0%


## PART 2

In [69]:
#LOADING IN DATA FOR PART 2

train_lines = []
with open('40.csv', encoding="utf8") as csv_file3:
    train_raw2 = csv.reader(csv_file3, delimiter=',')
    next(train_raw2)
    train_text = ''
    for line in train_raw2:
        train_text += str(line)
        
        train_lines.append(line[0])
train_text  = text_processor(train_text)


In [70]:
#Generating prior probabilitiies


vocabulary40_all = vocabulary_generate(train_text)
word_prior_probs = freq2probability(vocabulary40_all)




In [71]:
# GENERATING CLASS CONDITIONAL PROBABILITIES

word_lib = {}                                            #Here we loop through every single word to generate a dictionary                               
                                                         # corresponding to every word containing the class conditional 
for word in train_text:                                  # probabilites(finally as uber_lib) of all other words
    for i in range(len(train_lines)):
        if word not in word_lib:
            word_lib[word] = ''
        if word in (text_processor(train_lines[i])):
            word_lib[word] += train_lines[i]

uber_lib = library_constructor(word_lib)   #Here uber_lib serves a similar function to mega_lib earlier
                                           #It contains the class conditional probabilites of every word given another.

In [72]:
#LOADING IN THE TESTING SET

test_lines = []
with open('10.csv', encoding="utf8") as csv_file4:
    test_raw2 = csv.reader(csv_file4, delimiter=',')
    next(test_raw2)
    for line in test_raw2:
        test_lines.append(line[0])


In [73]:
#TABULATING OUR PREDICTIONS OVER THE TESTING SET

table_row2 =[]
table_rows2 = []


table_header2 = ['Sentence']
table_header2.append('Prediction')

for test_sentence in test_lines:
    
    table_row2 =[]
    
    table_row2.append(test_sentence)
    confidences2 = naive_classifier(vocabulary40_all, uber_lib,test_sentence)
    table_row2.append(max(confidences2, key =confidences2.get))
    table_rows2.append(table_row2)
    
data_frame2 = pd.DataFrame(table_rows2, columns = table_header2)
data_frame2


,Sentence,Prediction
0,Eating healthy food is important for maintainn...,midst
1,Following a healthy diet will boost your _____,healthi
2,"Avoid eating chemical additives, added sugars ...",ad
3,"Your diet should be rich of vitamins D,K, calc...",calcium
4,"The healthier the food you eat, the better you...",better
5,"Dehydration causes tiredness, low energy and h...",water
6,People with kidney disease should avoid eating...,much
7,We should avoid eating transfats. Eating healt...,food
8,Mindless eating is often caused by eating alon...,front
9,Eating more junk food will make you feel uncom...,junk


In [ ]:
s